In [ ]:

# Importação de módulos necessários
import requests
import pandas as pd
import numpy as np

# Função para converter identificadores de genes Illumina para Entrez

def illuminaToEntrez(geneId):
    """
    Função para converter um identificador de gene do formato Illumina para o formato Entrez.
    A conversão é feita utilizando uma API disponível em 'https://biit.cs.ut.ee/gprofiler/api/convert/convert/'.
    """
    r = requests.post(
    url='https://biit.cs.ut.ee/gprofiler/api/convert/convert/',
    json={
        'organism':'hsapiens',
        'target':'ENTREZGENE',
        'query':[geneId],
    }
    )
    return r.json()['result'][0]["name"]

#illuminaToEntrez("ILMN_1701127")

# Importar o conjunto de dados de expressão gênica de indivíduos sem cancro e transposta
df_clinical_patient_n = pd.read_csv('normals_ExpressionMatrix.txt', sep='\t')
#df_clinical_patient = df_clinical_patient.tail(-4)
df_clinical_patient_n=df_clinical_patient_n.transpose()

# Converte os nomes das colunas do dataframe (identificadores de genes) para o formato Entrez

print(df_clinical_patient_n.columns)
df_clinical_patient_n.head()
novosNomes=[]
for value in df_clinical_patient_n.columns:
    nome=illuminaToEntrez(value)
    novosNomes.append(nome)
df_clinical_patient_n.columns=novosNomes
df_clinical_patient_n.to_csv('dataset_preprocessed_n.csv')

# Remover colunas com 'None' nos nomes
df_clinical_patient_n = pd.read_csv('dataset_preprocessed_n.csv', sep=',', index_col=0)
columns_to_remove = [col for col in df_clinical_patient_n.columns if 'None' in col]
df_clinical_patient_n = df_clinical_patient_n.drop(columns=columns_to_remove)
df_clinical_patient_n

# Importar o conjunto de dados de expressão genetica  de indivíduos com cancro

comCancro= pd.read_csv('_data_mrna_agilent_microarray_zscores_ref_all_samples.txt' , sep='\t', header=0, index_col=0)
#df_clinical_patient = df_clinical_patient.tail(-4)
comCancro=comCancro.transpose()
row=comCancro.iloc[0].astype(int)

# Sincronizar as colunas entre os dois dataframes (com cancro e sem cancro)
# Manter apenas as colunas que existem em ambos os conjuntos de dados

comCancro.iloc[0]=row
comCancro.head()
columns_to_remove = [col for col in df_clinical_patient_n.columns if col in comCancro.columns]
df_clinical_patient_n = df_clinical_patient_n[columns_to_remove]

# Adicionar uma coluna "hasCancer" para marcar se um indivíduo tem cancro ou não

df_clinical_patient_n["hasCancer"]=0
comCancro["hasCancer"]=1
comCancro=comCancro.iloc[1:]

# Normalizar os dados de expressão genética do conjunto sem cancro com z-score 
aux=df_clinical_patient_n["hasCancer"]
df_clinical_patient_n.drop(["hasCancer"],axis=1)
df_clinical_patient_n_2 =  (df_clinical_patient_n - df_clinical_patient_n.mean()) / df_clinical_patient_n.std()
df_clinical_patient_n_2["hasCancer"]=aux

# Combinar os conjuntos de dados normalizados e remover quaisquer colunas com valores NA porque mesmo a eliminar os none havia genes que estavam num dataset e no outro era none
final5=pd.concat([df_clinical_patient_n_2,comCancro])
final5=final5.dropna(axis=1)
final5.to_csv("final_n.csv")
